# 서울교통공사_역별시간대별혼잡도
- 1-8호선
- https://data.seoul.go.kr/dataList/OA-12928/F/1/datasetView.do
- 정원대비 승차인원으로, 승차인과 좌석수가 일치할 경우를 혼잡도 34%로 산정 
- 34% (좌석수 54)
- 100% (160명, 통로3줄, 각 출입문 2명)
- 150% (240명)
- api(분기별 업데이트) : https://www.data.go.kr/data/15071311/fileData.do#tab-layer-openapi

In [3]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime
import os
import csv
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
pop = {'34%':'54명', '100%':'160명', '150%':'240명'}

In [5]:
URL = 'https://api.odcloud.kr/api/15071311/v1/uddi:da7cd08f-94f0-4dba-b33d-d02dcb35b57b?page=1&perPage=1000&returnType=json&serviceKey=PGcAcF%2BY%2B7d8GehyJRxrnwmedSRiOcvC%2F%2FXE8cVtOxlB4G9V68Lbu18Ga1CBdUkemypvosbtQHyQEjpMxRiEZw%3D%3D'

page = requests.get(URL)
data = page.json()

print(page.status_code)
#print(page.json)

200


In [6]:
cols = ['00시00분', '00시30분', '10시00분', '10시30분', '11시00분', '11시30분', '12시00분', '12시30분', '13시00분', 
        '13시30분', '14시00분', '14시30분', '15시00분', '15시30분', '16시00분', '16시30분', '17시00분', '17시30분', 
        '18시00분', '18시30분', '19시00분', '19시30분', '20시00분', '20시30분', '21시00분', '21시30분', '22시00분', 
        '22시30분', '23시00분', '23시30분', '5시30분', '6시00분', '6시30분', '7시00분', '7시30분', '8시00분', '8시30분', 
        '9시00분', '9시30분', '상하구분', '역번호', '요일구분', '출발역', '호선']

val_list = []
for i in data['data']:
    #print(i.keys())
    val_list.append(list(i.values()))

In [7]:
subwayDF = pd.DataFrame(val_list, columns=cols)
subwayDF.head()

,00시00분,00시30분,10시00분,10시30분,11시00분,11시30분,12시00분,12시30분,13시00분,13시30분,...,7시30분,8시00분,8시30분,9시00분,9시30분,상하구분,역번호,요일구분,출발역,호선
0,8.70,0.0,15.30,15.50,16.20,14.10,14.70,16.30,19.0,17.0,...,10.30,17.50,14.40,11.70,15.40,상선,158,평일,청량리,1
1,2.40,0.2,34.30,27.20,18.90,24.00,23.60,21.70,23.9,22.5,...,70.20,66.60,63.00,48.80,34.50,하선,158,평일,청량리,1
2,11.90,0.0,15.10,19.10,20.80,16.20,17.50,19.50,22.7,22.3,...,12.50,20.10,17.60,15.80,23.90,상선,157,평일,제기동,1
3,2.30,0.3,35.00,27.80,24.80,22.20,27.30,26.70,23.1,26.1,...,63.20,74.10,64.50,50.30,35.30,하선,157,평일,제기동,1
4,13.10,0.0,19.80,22.60,25.50,19.80,20.00,24.80,27.0,26.7,...,14.50,21.70,20.90,19.40,29.00,상선,156,평일,신설동,1


In [9]:
subwayDF['역구분'] = subwayDF['출발역']+ '_' +subwayDF['상하구분']

In [10]:
subwayDF['max'] = subwayDF[cols].max(axis=1)

In [ ]:
subwayDF

,00시00분,00시30분,10시00분,10시30분,11시00분,11시30분,12시00분,12시30분,13시00분,13시30분,...,8시30분,9시00분,9시30분,상하구분,역번호,요일구분,출발역,호선,역구분,max
0,8.70,0.0,15.30,15.50,16.20,14.10,14.70,16.30,19.0,17.0,...,14.40,11.70,15.40,상선,158,평일,청량리,1,청량리_상선,158
1,2.40,0.2,34.30,27.20,18.90,24.00,23.60,21.70,23.9,22.5,...,63.00,48.80,34.50,하선,158,평일,청량리,1,청량리_하선,158
2,11.90,0.0,15.10,19.10,20.80,16.20,17.50,19.50,22.7,22.3,...,17.60,15.80,23.90,상선,157,평일,제기동,1,제기동_상선,157
3,2.30,0.3,35.00,27.80,24.80,22.20,27.30,26.70,23.1,26.1,...,64.50,50.30,35.30,하선,157,평일,제기동,1,제기동_하선,157
4,13.10,0.0,19.80,22.60,25.50,19.80,20.00,24.80,27.0,26.7,...,20.90,19.40,29.00,상선,156,평일,신설동,1,신설동_상선,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.00,None,2.90,2.60,2.50,2.40,2.30,2.00,2.5,2.5,...,3.10,3.40,2.80,하선,2711,토요일,장암,7,장암_하선,2711
996,0.00,None,0.90,1.10,1.70,1.20,1.10,1.20,1.1,0.5,...,1.20,0.80,2.10,상선,2712,토요일,도봉산,7,도봉산_상선,2712
997,0.00,None,16.20,16.60,12.40,14.80,14.20,15.70,13.0,14.3,...,18.60,15.70,16.70,하선,2712,토요일,도봉산,7,도봉산_하선,2712
998,5.80,None,14.40,12.40,12.90,11.00,12.00,11.30,10.8,11.8,...,11.00,12.50,16.00,상선,2713,토요일,수락산,7,수락산_상선,2713
